# workflow: converting old Morpheus benchmark to ReMorpheus format

### step1: compose `example.json`
- see an example in `benchmarks/morpheus/5/example.json`
- working directory is `benchmarks/morpheus/`

### step2: compose solution program and test it out
- see the original Morpheus logs (`tests/morpheus-logs.tar`) for reference solution (note that 1. the reference solution may not generate the same output in ReMorpheus system; 2. the new solution may even be of a different size then the old Morpheus one)
- attach `tag={"cpos":0}` to trick the interpreter, since naturally generated program contains such information from the enumerator; but here we do not need it since that's for deduction purpose

In [1]:
import pandas as pd
import json

import tyrell.spec as S
import tyrell.dsl as D
from tyrell.interpreter import MorpheusInterpreter
from tyrell.dsl.node import AtomNode, ParamNode, ApplyNode, Mutable

from test_morpheus_benchmark import load_example

def tag_trick(arg_node, ind=Mutable(0)):
    if isinstance(arg_node, AtomNode):
        arg_node._tag = {"cpos":ind.v}
        ind.v += 1
    elif isinstance(arg_node, ParamNode):
        pass
    elif isinstance(arg_node, ApplyNode):
        for p in arg_node.args:
            tag_trick(p)

In [2]:
spec = S.parse_file("./dsls/morpheus.tyrell")
builder = D.Builder(spec)
interpreter = MorpheusInterpreter(spec=spec)

In [3]:
# construct: gather(@param0, ['-99', '-1'])
step0_func_prod = spec.get_function_production("gather")
step0_arg_nodes = [
    builder.make_node( spec.get_param_production(0) ),
    builder.make_node( spec.get_enum_production(spec.get_type("ColList"),['-99', '-1']) )
]
step0_node = builder.make_node( step0_func_prod, step0_arg_nodes )

In [4]:
# construct: separate(gather(@param0, ['-99', '-1']), 2)
step1_func_prod = spec.get_function_production("separate")
step1_arg_nodes = [
    step0_node,
    builder.make_node( spec.get_enum_production(spec.get_type("ColInt"),"2") )
]
step1_node = builder.make_node( step1_func_prod, step1_arg_nodes )

In [5]:
# construct: select(separate(gather(@param0, ['-99', '-1']), 2), ['-3'])
step2_func_prod = spec.get_function_production("select")
step2_arg_nodes = [
    step1_node,
    builder.make_node( spec.get_enum_production(spec.get_type("ColList"),['-3']) )
]
step2_node = builder.make_node( step2_func_prod, step2_arg_nodes )

In [7]:
program = step2_node
tag_trick(program)
print(program)

select(separate(gather(@param0, ['-99', '-1']), 2), ['-3'])


In [8]:
# load example from benchmark 5
example = load_example(5)

In [9]:
# trick the interpreter, need to be resetted every time before you call `eval`
interpreter._current_combination = tuple([None for _ in range(30)])
actual_output = interpreter.eval( program, example.input )
actual_output

,ID,T,COL1,COL3
0,1,24.3,10.2,1
1,2,23.4,10.4,1
2,1,24.3,5.5,2
3,2,23.4,5.7,2
4,1,24.3,4.5,1
5,2,23.4,3.2,1


In [10]:
# compare and verify the correctness of the program
if interpreter.equal_tb(actual_output, example.output) is None:
    print("good to go")
else:
    print("will throw some exception if not good")

good to go


In [11]:
# then add this to the "solution" field of `example.json`
program.to_jsonexp()

[['function', 'select'],
 [['function', 'separate'],
  [['function', 'gather'], ['param', 0], ['enum', 'ColList', ['-99', '-1']]],
  ['enum', 'ColInt', '2']],
 ['enum', 'ColList', ['-3']]]

### step2 (alternative): compose solution program from json expression

In [12]:
jsonexp = [
    ["function", "select"],
    [
        ["function", "separate"],
        [
            ["function", "gather"],
            ["param", 0],
            ["enum", "ColList", ["-99", "-1"]]
        ],
        ["enum", "ColInt", "2"]
    ],
    ["enum", "ColList", ["-3"]]
]

In [15]:
program = builder.from_jsonexp(jsonexp)
tag_trick(program)

In [16]:
# trick the interpreter, need to be resetted every time before you call `eval`
interpreter._current_combination = tuple([None for _ in range(30)])
actual_output = interpreter.eval( program, example.input )
actual_output

,ID,T,COL1,COL3
0,1,24.3,10.2,1
1,2,23.4,10.4,1
2,1,24.3,5.5,2
3,2,23.4,5.7,2
4,1,24.3,4.5,1
5,2,23.4,3.2,1


In [17]:
if interpreter.equal_tb(actual_output, example.output) is None:
    print("good to go")
else:
    print("will throw some exception if not good")

good to go


### step3: sort out folder and add records
- when the previous steps are done, sort out the folder by only keeping the `example.json` (e.g., removing all the `csv` files)
- keep track of the `translation status` in the google sheet: [https://docs.google.com/spreadsheets/d/1ofA7EQKic4U-KOeLcFiUWbrPUwfjKe0SzgyRUKsyudo/edit?usp=sharing](https://docs.google.com/spreadsheets/d/1ofA7EQKic4U-KOeLcFiUWbrPUwfjKe0SzgyRUKsyudo/edit?usp=sharing)